In [ ]:
from google.colab import drive
drive.mount("/content/myDrive")

Drive already mounted at /content/myDrive; to attempt to forcibly remount, call drive.mount("/content/myDrive", force_remount=True).


In [ ]:
import torch 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
use_cuda = torch.cuda.is_available()
print(use_cuda)
if use_cuda:
  print(torch.cuda.get_device_name(0))

cuda:0
True
Tesla T4


In [ ]:
#-- 3.8.16
!wget https://www.python.org/ftp/python/3.8.16/Python-3.8.16.tgz
!tar xvfz Python-3.8.16.tgz
!Python-3.8.16/configure
!sudo make install
!python __version

In [ ]:
!pip install scrapbook
!pip install recommenders
#-- https://github.com/microsoft/recommenders

In [ ]:
!pip install LibRecommender
#-- https://github.com/massquantity/LibRecommender#references 라이브러리

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.5 MB/s eta 0:00:00


In [ ]:
# Install required packages.
# !pip install torch-sparse
print(torch.__version__)
print(torch.version.cuda)

!pip install torch-geometric
!pip install -q git+https://github.com/snap-stanford/deepsnap.git
"""
DeepSNAP는 그래프 신경망(Graph Neural Networks)을 구축하고 훈련하기 위한 파이썬 패키지
DeepSNAP은 PyTorch 기반의 그래프 신경망 라이브러리 PyG(PyTorch Geometric)에서 영감을 받아 개발되었다. DeepSNAP은 PyG와 유사한 인터페이스를 제공하여 그래프 데이터를 로드하고 전처리할 수 있다.
"""
!pip install -U -q PyDrive
!pip install torch_scatter -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cu118.html

2.0.1+cu118
11.8
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=96b05b3ff9d7f0d5b69951d1c58e983560c4d7a073f98bda62c4e055691289cb
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 11.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev

## 필요 라이브러리 Import


In [ ]:
import numpy as np
import pandas as pd
from libreco.data import random_split, DatasetPure
from libreco.algorithms import LightGCN  # pure data, algorithm NGCF
from libreco.evaluation import evaluate
#-- Libreco

import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch import nn, optim, Tensor
from torch_sparse import SparseTensor, matmul
from torch_geometric.utils import structured_negative_sampling
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import Adj
#-- import required modules Blog

## 데이터 준비 movielens 10만개 데이터 

In [ ]:
# download the dataset
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
extract_zip(download_url(url, '.'), '.')

rating_path = '/content/ml-latest-small/ratings.csv'
item_path = "/content/ml-latest-small/movies.csv"

Using existing file ml-latest-small.zip
Extracting ./ml-latest-small.zip


In [ ]:

def libco_LightGCN_data(path, data_frame, ratio = [0.8, 0.1, 0.1], excluded_user=None):
  #-- data
  data = pd.read_csv(path)
  data.rename(columns = {data_frame[0] : "user", data_frame[1]:"item", data_frame[2]:"label",},
              inplace = True)
  
  # print(data.head(10))
  #-- 테스트를 위한 특정 유저의 가장 앞에 있는 데이터 n개 삭제
  if excluded_user is not None:
    user_data = data[data["user"] == excluded_user]
    user_data_5stars = user_data[user_data["label"] == 5.0]  # Filter the data with a rating of 5.0
    if not user_data_5stars.empty:
      excluded_number = 10
      user_data_5stars = user_data_5stars.head(excluded_number)
      data = data.drop(user_data_5stars.index)

  # print(data.head(10))

  # split whole data into three folds for training, evaluating and testing
  train_data, eval_data, test_data = random_split(data, multi_ratios= ratio)

  train_data, data_info = DatasetPure.build_trainset(train_data)
  eval_data = DatasetPure.build_evalset(eval_data)
  test_data = DatasetPure.build_testset(test_data)
  tet_data = [train_data, eval_data, test_data]
  print(data_info)  # n_users: 5894, n_items: 3253, data sparsity: 0.4172 %  
  return tet_data, data_info
  #-- Libco -- 

def blog_LightGCN_data(path, data_frame, rating_threshold =4, excluded_user=None):

  #-- 유저, 아이템 매핑
  rating_data = pd.read_csv(path[0], index_col= data_frame[0])
  item_data = pd.read_csv(path[1], index_col= data_frame[1])
  data = pd.read_csv(path[0])

  print("1번째 데이터 삭제하기 전")
  print(rating_data.head(10))

  #-- 테스트를 위한 특정 유저의 가장 앞에 있는 데이터 삭제

  if excluded_user is not None:
    user_data = data[data[data_frame[0]] == excluded_user]
    user_data_5stars = user_data[user_data[data_frame[2]] == 5.0]  # Filter the data with a rating of 5.0r_user_data[r_user_data[data_frame[2]] == 5.0]  # Filter the data with a rating of 5.0

    r_user_data =  rating_data[rating_data.index == excluded_user].reset_index()

    if not user_data_5stars.empty:
      excluded_number = 10
      user_data_5stars = user_data_5stars.head(excluded_number)     
      r_user_data_5stars = r_user_data[r_user_data[data_frame[2]] == 5.0]
      data = data.drop(user_data_5stars.index)
      
      row_index = []
      for i in range(0,excluded_number):
        row_index.append(r_user_data_5stars.iloc[i].name)

      rating_data = rating_data.reset_index().drop(row_index[0])
      rating_data.set_index(data_frame[0], inplace=True)
      for i in range(1, len(row_index)) :
        rating_data = rating_data.reset_index().drop(row_index[i]-i)
      # rating_data = rating_data.drop(rating_data.iloc.nam)
        rating_data.set_index(data_frame[0], inplace=True)
  
  print("\n\n\n1번째 데이터 삭제하기 후")
  print(rating_data.head(10))

  # # rating_data의 컬럼 확인
  # print(rating_data.head(5))

  user_mapping = {index: i for i, index in enumerate(rating_data.index.unique())}
  item_mapping = {index: i for i, index in enumerate(item_data.index.unique())}

  #-- 매핑 데이터로 edge연결
  edge_index = None
  src = [user_mapping[index] for index in data[data_frame[0]]]
  dst = [item_mapping[index] for index in data[data_frame[1]]]
  edge_attr = torch.from_numpy(data[data_frame[2]].values).view(-1, 1).to(torch.long) >= rating_threshold
  
  edge_index = [[], []]
  for i in range(edge_attr.shape[0]):
    if edge_attr[i]:
        edge_index[0].append(src[i])
        edge_index[1].append(dst[i])
  edge_index = torch.tensor(edge_index)

  #-- 데이터 분할 8:1:1
  num_users, num_movies = len(user_mapping), len(item_mapping)
  num_interactions = edge_index.shape[1]
  all_indices = [i for i in range(num_interactions)]

  train_indices, test_indices = train_test_split(
      all_indices, test_size=0.2, random_state=1)
  val_indices, test_indices = train_test_split(
      test_indices, test_size=0.5, random_state=1)

  train_edge_index = edge_index[:, train_indices]
  val_edge_index = edge_index[:, val_indices]
  test_edge_index = edge_index[:, test_indices]

  print("Train Edge Data : ", train_edge_index.shape)
  print("Val Edge Data :", val_edge_index.shape)
  print("Test Edge Data : ", test_edge_index.shape)    

  #-- 연결된 엣지를 희소행렬로 변환
  # convert edge indices into Sparse Tensors: https://pytorch-geometric.readthedocs.io/en/latest/notes/sparse_tensor.html
  from torch_geometric.utils import train_test_split_edges

  train_sparse_edge_index = SparseTensor(row=train_edge_index[0], col=train_edge_index[1], sparse_sizes=(
      num_users + num_movies, num_users + num_movies))
  val_sparse_edge_index = SparseTensor(row=val_edge_index[0], col=val_edge_index[1], sparse_sizes=(
      num_users + num_movies, num_users + num_movies))
  test_sparse_edge_index = SparseTensor(row=test_edge_index[0], col=test_edge_index[1], sparse_sizes=(
      num_users + num_movies, num_users + num_movies))

  tet_edge_data = [edge_index, train_edge_index,val_edge_index, test_edge_index]  
  tet_sparse_data = [train_sparse_edge_index, val_sparse_edge_index, test_sparse_edge_index]
  return tet_edge_data, tet_sparse_data, user_mapping, item_mapping
#-- Blog -- 

In [ ]:
excluded_user_id = 1
data_frame = ["userId", "movieId", "rating"]
path = [rating_path, item_path]

lib_tet_data, lib_data_info = libco_LightGCN_data(path[0], data_frame, excluded_user = excluded_user_id)
blog_tet_edge_data, blog_tet_sparse_data, user_mapping, item_mapping = blog_LightGCN_data(path, data_frame, 4, excluded_user_id)

n_users: 610, n_items: 8967, data density: 1.4747 %
1번째 데이터 삭제하기 전
        movieId  rating  timestamp
userId                            
1             1     4.0  964982703
1             3     4.0  964981247
1             6     4.0  964982224
1            47     5.0  964983815
1            50     5.0  964982931
1            70     3.0  964982400
1           101     5.0  964980868
1           110     4.0  964982176
1           151     5.0  964984041
1           157     5.0  964984100



1번째 데이터 삭제하기 후
        movieId  rating  timestamp
userId                            
1             1     4.0  964982703
1             3     4.0  964981247
1             6     4.0  964982224
1            70     3.0  964982400
1           110     4.0  964982176
1           151     5.0  964984041
1           157     5.0  964984100
1           163     5.0  964983650
1           216     5.0  964981208
1           223     3.0  964980985
Train Edge Data :  torch.Size([2, 38861])
Val Edge Data : torch.Size([2, 48

## 모델 정의

In [ ]:
def libco_LightGCN(data_info, embed_size =32, n_epochs = 20):
  lightgcn = LightGCN(
      task="ranking",
      data_info= data_info,
      loss_type="bpr",
      embed_size= embed_size,
      n_epochs=n_epochs,
      lr=1e-3,
      batch_size=2048,
      num_neg=1,
      device="cuda",
  )
  return lightgcn
#-- Libco --#

class Blog_LightGCN(MessagePassing):
      def __init__(self, num_users, num_items, embedding_dim=64, K=3, add_self_loops=False):
          super().__init__()
          self.num_users, self.num_items = num_users, num_items
          self.embedding_dim, self.K = embedding_dim, K
          self.add_self_loops = add_self_loops
          self.users_emb = nn.Embedding(
              num_embeddings=self.num_users, embedding_dim=self.embedding_dim) # e_u^0
          self.items_emb = nn.Embedding(
              num_embeddings=self.num_items, embedding_dim=self.embedding_dim) # e_i^0
          nn.init.normal_(self.users_emb.weight, std=0.1)
          nn.init.normal_(self.items_emb.weight, std=0.1)
      def forward(self, edge_index: SparseTensor):
          # compute \tilde{A}: symmetrically normalized adjacency matrix
          edge_index_norm = gcn_norm(
              edge_index, add_self_loops=self.add_self_loops)
          emb_0 = torch.cat([self.users_emb.weight, self.items_emb.weight]) # E^0
          embs = [emb_0]
          emb_k = emb_0

          # multi-scale diffusion
          for i in range(self.K):
              emb_k = self.propagate(edge_index_norm, x=emb_k)
              embs.append(emb_k)
          embs = torch.stack(embs, dim=1)
          emb_final = torch.mean(embs, dim=1) # E^K
          users_emb_final, items_emb_final = torch.split(
              emb_final, [self.num_users, self.num_items]) # splits into e_u^K and e_i^K
          # returns e_u^K, e_u^0, e_i^K, e_i^0
          return users_emb_final, self.users_emb.weight, items_emb_final, self.items_emb.weight

      def message(self, x_j: Tensor) -> Tensor:
          return x_j

      def message_and_aggregate(self, adj_t: SparseTensor, x: Tensor) -> Tensor:
          # computes \tilde{A} @ x
          return matmul(adj_t, x)

#-- 손실함수 bpr
def bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, lambda_val):
    reg_loss = lambda_val * (users_emb_0.norm(2).pow(2) +
                             pos_items_emb_0.norm(2).pow(2) +
                             neg_items_emb_0.norm(2).pow(2)) # L2 loss

    pos_scores = torch.mul(users_emb_final, pos_items_emb_final)
    pos_scores = torch.sum(pos_scores, dim=-1) # predicted scores of positive samples
    neg_scores = torch.mul(users_emb_final, neg_items_emb_final)
    neg_scores = torch.sum(neg_scores, dim=-1) # predicted scores of negative samples

    loss = -torch.mean(torch.nn.functional.softplus(pos_scores - neg_scores)) + reg_loss
    return loss

def blog_LightGCN(num_users, num_movies, embed_size):
  # defines LightGCN model
  model = Blog_LightGCN(num_users, num_movies, embedding_dim = embed_size)
  return model

#-- Blog --#

In [ ]:
lib_model = libco_LightGCN(lib_data_info, embed_size =32, n_epochs = 20) 
blog_model = blog_LightGCN(len(user_mapping), len(item_mapping), embed_size= 32)

### 모델 학습

In [ ]:
def libco_fit(model, train_data, eval_data):
  model.fit(
        train_data,
        neg_sampling=True,
        verbose=1,
        eval_data=eval_data,
        metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
    )
#-- Libco--#


# function which random samples a mini-batch of positive and negative samples
def sample_mini_batch(batch_size, edge_index):
    edges = structured_negative_sampling(edge_index)
    edges = torch.stack(edges, dim=0)
    indices = random.choices(
        [i for i in range(edges[0].shape[0])], k=batch_size)
    batch = edges[:, indices]
    user_indices, pos_item_indices, neg_item_indices = batch[0], batch[1], batch[2]
    return user_indices, pos_item_indices, neg_item_indices

# helper function to get N_u
def get_user_positive_items(edge_index):
    user_pos_items = {}
    for i in range(edge_index.shape[1]):
        user = edge_index[0][i].item()
        item = edge_index[1][i].item()
        if user not in user_pos_items:
            user_pos_items[user] = []
        user_pos_items[user].append(item)
    return user_pos_items

# computes recall@K and precision@K
def RecallPrecision_ATk(groundTruth, r, k):
    num_correct_pred = torch.sum(r, dim=-1)  # number of correctly predicted items per user
    # number of items liked by each user in the test set
    user_num_liked = torch.Tensor([len(groundTruth[i])
                                  for i in range(len(groundTruth))])
    recall = torch.mean(num_correct_pred / user_num_liked)
    precision = torch.mean(num_correct_pred) / k
    return recall.item(), precision.item()

# computes NDCG@K
def NDCGatK_r(groundTruth, r, k):
    assert len(r) == len(groundTruth)

    test_matrix = torch.zeros((len(r), k))

    for i, items in enumerate(groundTruth):
        length = min(len(items), k)
        test_matrix[i, :length] = 1
    max_r = test_matrix
    idcg = torch.sum(max_r * 1. / torch.log2(torch.arange(2, k + 2)), axis=1)
    dcg = r * (1. / torch.log2(torch.arange(2, k + 2)))
    dcg = torch.sum(dcg, axis=1)
    idcg[idcg == 0.] = 1.
    ndcg = dcg / idcg
    ndcg[torch.isnan(ndcg)] = 0.
    return torch.mean(ndcg).item()

# wrapper function to get evaluation metrics
def get_metrics(model, edge_index, exclude_edge_indices, k):
    user_embedding = model.users_emb.weight
    item_embedding = model.items_emb.weight

    # get ratings between every user and item - shape is num users x num movies
    rating = torch.matmul(user_embedding, item_embedding.T)

    for exclude_edge_index in exclude_edge_indices:
        # gets all the positive items for each user from the edge index
        user_pos_items = get_user_positive_items(exclude_edge_index)
        # get coordinates of all edges to exclude
        exclude_users = []
        exclude_items = []
        for user, items in user_pos_items.items():
            exclude_users.extend([user] * len(items))
            exclude_items.extend(items)

        # set ratings of excluded edges to large negative value
        rating[exclude_users, exclude_items] = -(1 << 10)

    # get the top k recommended items for each user
    _, top_K_items = torch.topk(rating, k=k)

    # get all unique users in evaluated split
    users = edge_index[0].unique()

    test_user_pos_items = get_user_positive_items(edge_index)

    # convert test user pos items dictionary into a list
    test_user_pos_items_list = [
        test_user_pos_items[user.item()] for user in users]

    # determine the correctness of topk predictions
    r = []
    for user in users:
        ground_truth_items = test_user_pos_items[user.item()]
        label = list(map(lambda x: x in ground_truth_items, top_K_items[user]))
        r.append(label)
    r = torch.Tensor(np.array(r).astype('float'))

    recall, precision = RecallPrecision_ATk(test_user_pos_items_list, r, k)
    ndcg = NDCGatK_r(test_user_pos_items_list, r, k)

    return recall, precision, ndcg
# wrapper function to evaluate model
def evaluation(model, edge_index, sparse_edge_index, exclude_edge_indices, k, lambda_val):
    # get embeddings
    users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(
        sparse_edge_index)
    edges = structured_negative_sampling(
        edge_index, contains_neg_self_loops=False)
    user_indices, pos_item_indices, neg_item_indices = edges[0], edges[1], edges[2]
    users_emb_final, users_emb_0 = users_emb_final[user_indices], users_emb_0[user_indices]
    pos_items_emb_final, pos_items_emb_0 = items_emb_final[
        pos_item_indices], items_emb_0[pos_item_indices]
    neg_items_emb_final, neg_items_emb_0 = items_emb_final[
        neg_item_indices], items_emb_0[neg_item_indices]

    loss = bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0,
                    neg_items_emb_final, neg_items_emb_0, lambda_val).item()

    recall, precision, ndcg = get_metrics(
        model, edge_index, exclude_edge_indices, k)

    return loss, recall, precision, ndcg
def blog_fit(model, blog_tet_edge_data, blog_tet_sparse_data, epochs = 50):
  # define contants
  ITERATIONS = epochs
  BATCH_SIZE = 2048
  LR = 1e-3
  ITERS_PER_EVAL = 300
  ITERS_PER_LR_DECAY = 300
  K = 10
  LAMBDA = 1e-6

  model = model.to(device)
  model.train()

  optimizer = optim.Adam(model.parameters(), lr=LR)
  scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

  edge_index = blog_tet_edge_data[0].to(device)
  train_edge_index = blog_tet_edge_data[1].to(device)
  train_sparse_edge_index = blog_tet_sparse_data[0].to(device)

  val_edge_index = blog_tet_edge_data[2].to(device)
  val_sparse_edge_index = blog_tet_sparse_data[1].to(device)

  # training loop
  train_losses = []
  val_losses = []

  for iter in range(ITERATIONS):
      # forward propagation
      users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(
          train_sparse_edge_index)

      # mini batching
      user_indices, pos_item_indices, neg_item_indices = sample_mini_batch(
          BATCH_SIZE, train_edge_index)
      user_indices, pos_item_indices, neg_item_indices = user_indices.to(
          device), pos_item_indices.to(device), neg_item_indices.to(device)
      users_emb_final, users_emb_0 = users_emb_final[user_indices], users_emb_0[user_indices]
      pos_items_emb_final, pos_items_emb_0 = items_emb_final[
          pos_item_indices], items_emb_0[pos_item_indices]
      neg_items_emb_final, neg_items_emb_0 = items_emb_final[
          neg_item_indices], items_emb_0[neg_item_indices]

      # loss computation
      train_loss = bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final,
                            pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, LAMBDA)

      optimizer.zero_grad()
      train_loss.backward()
      optimizer.step()

      if iter % ITERS_PER_EVAL == 0:
          model.eval()
          val_loss, recall, precision, ndcg = evaluation(
              model, val_edge_index, val_sparse_edge_index, [train_edge_index], K, LAMBDA)
          print(f"[Iteration {iter}/{ITERATIONS}] train_loss: {round(train_loss.item(), 5)}, val_loss: {round(val_loss, 5)}, val_recall@{K}: {round(recall, 5)}, val_precision@{K}: {round(precision, 5)}, val_ndcg@{K}: {round(ndcg, 5)}")
          train_losses.append(train_loss.item())
          val_losses.append(val_loss)
          model.train()

      if iter % ITERS_PER_LR_DECAY == 0 and iter != 0:
          scheduler.step()
  iters = [iter * ITERS_PER_EVAL for iter in range(len(train_losses))]
  plt.plot(iters, train_losses, label='train')
  plt.plot(iters, val_losses, label='validation')
  plt.xlabel('iteration')
  plt.ylabel('loss')
  plt.title('training and validation loss curves')
  plt.legend()
  plt.show()

In [ ]:
libco_fit(lib_model, lib_tet_data[0], lib_tet_data[1])
blog_fit(blog_model, blog_tet_edge_data, blog_tet_sparse_data, epochs =10000)

## 모델평가

In [ ]:
def libco_eval(model, test_data):
  result = evaluate(
      model=model,
      data=test_data,
      neg_sampling=True,
      metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
  )
  print(result)

#-- Libco --

def blog_eval(model, blog_tet_edge_data, blog_tet_sparse_data , K = 10):
  # evaluate on test set
  LAMBDA = 1e-6
  model.eval()
  test_edge_index = blog_tet_edge_data[3].to(device)
  test_sparse_edge_index = blog_tet_sparse_data[2].to(device)

  test_loss, test_recall, test_precision, test_ndcg = evaluation(
              model, test_edge_index, test_sparse_edge_index, [blog_tet_edge_data[1], blog_tet_edge_data[2]], K, LAMBDA)

  print(f"[test_loss: {round(test_loss, 5)}, test_recall@{K}: {round(test_recall, 5)}, test_precision@{K}: {round(test_precision, 5)}, test_ndcg@{K}: {round(test_ndcg, 5)}")


In [ ]:
libco_eval(lib_model, lib_tet_data[2])
blog_eval(blog_model, blog_tet_edge_data, blog_tet_sparse_data, 10)

### 모델 예측

In [ ]:
def lilco_predict(model, user_id, path, K = 10):
  df = pd.read_csv(path[1])
  rec_arr = model.recommend_user(user=user_id, n_rec=K).get(user_id)
  
  titles = []
  genres = []
  for i in rec_arr:
    title = df[df['movieId'] == i]['title']
    genre = df[df['movieId'] == i]['genres']
    titles.append(title)
    genres.append(genre)
    # print(f"title: {title}, genres: {genre} ")


  return titles, genres
#-- Libco  --#

def blog_predict(model, user_id ,TopK, path, user_mapping, item_mapping, edge_index):

  df = pd.read_csv(path[1])
  movieid_title = pd.Series(df.title.values,index=df.movieId).to_dict()
  movieid_genres = pd.Series(df.genres.values,index=df.movieId).to_dict()

  user_pos_items = get_user_positive_items(edge_index)
  user = user_mapping[user_id]
  e_u = model.users_emb.weight[user]
  scores = model.items_emb.weight @ e_u

  values, indices = torch.topk(scores, k=len(user_pos_items[user]) + TopK)

  movies = [index.cpu().item() for index in indices if index in user_pos_items[user]][:TopK]
  movie_ids = [list(item_mapping.keys())[list(item_mapping.values()).index(movie)] for movie in movies]
  titles = [movieid_title[id] for id in movie_ids]
  genres = [movieid_genres[id] for id in movie_ids]
 
  # print(f"Here are some movies that user {user_id} rated highly")
  # for i in range(TopK):
  #     print(f"title: {titles[i]}, genres: {genres[i]} ")

  movies = [index.cpu().item() for index in indices if index not in user_pos_items[user]][:TopK]
  movie_ids = [list(item_mapping.keys())[list(item_mapping.values()).index(movie)] for movie in movies]
  titles = [movieid_title[id] for id in movie_ids]
  genres = [movieid_genres[id] for id in movie_ids]

  # print(f"Here are some suggested movies for user {user_id}")
  return titles, genres

In [ ]:
user_id = 1
K = 5
lilco_predict(lib_model,user_id, path, K = K)
blog_predict(blog_model, user_id , K, path, user_mapping, item_mapping, blog_tet_edge_data[0])

## 프레임워크

In [ ]:
def libco(data_frame, path, embed_size, n_epochs, excluded_user):
  lib_tet_data, lib_data_info = libco_LightGCN_data(path[0], data_frame, excluded_user = excluded_user_id)
  lib_model = libco_LightGCN(lib_data_info, embed_size =embed_size, n_epochs = int(n_epochs/3)) 
  libco_fit(lib_model, lib_tet_data[0], lib_tet_data[1])
  libco_eval(lib_model, lib_tet_data[2])
  titles, genres = lilco_predict(lib_model,user_id, path, K = K)
  return titles,genres
  
def blog(data_frame, path, embed_size, n_epochs, excluded_user):
  blog_tet_edge_data, blog_tet_sparse_data, user_mapping, item_mapping = blog_LightGCN_data(path, data_frame, 4, excluded_user_id)
  blog_model = blog_LightGCN(len(user_mapping), len(item_mapping), embed_size= 32)
  blog_fit(blog_model, blog_tet_edge_data, blog_tet_sparse_data, epochs =n_epochs)
  blog_eval(blog_model, blog_tet_edge_data, blog_tet_sparse_data, 10)
  titles, genres = blog_predict(blog_model, user_id , K, path, user_mapping, item_mapping, blog_tet_edge_data[0])
  return titles,genres

In [ ]:
data_frame = ["userId", "movieId", "rating"]
path = [rating_path, item_path]
embed_size = 16
n_epochs = 20000
user_id = 1
excluded_user = 1
K = 15


lib_titles, lib_genres = libco(data_frame, path,embed_size, n_epochs, excluded_user)
blog_titles, blog_genres = blog(data_frame, path, embed_size, n_epochs, excluded_user)

In [ ]:
for i in range(K):
      print(f"title: {lib_titles[i].values}, genres: {lib_genres[i].values} ")
print("========================================================")
for i in range(K):
      print(f"title: {blog_titles[i]}, genres: {blog_genres[i]} ")      


title: ['Absent-Minded Professor, The (1961)'], genres: ['Children|Comedy|Fantasy'] 
title: ['NeverEnding Story, The (1984)'], genres: ['Adventure|Children|Fantasy'] 
title: ['NeverEnding Story III, The (1994)'], genres: ['Adventure|Children|Fantasy'] 
title: ['Friday the 13th (1980)'], genres: ['Horror|Mystery|Thriller'] 
title: ['Indiana Jones and the Temple of Doom (1984)'], genres: ['Action|Adventure|Fantasy'] 
title: ['Fantasia (1940)'], genres: ['Animation|Children|Fantasy|Musical'] 
title: ['Aliens (1986)'], genres: ['Action|Adventure|Horror|Sci-Fi'] 
title: ['Balto (1995)'], genres: ['Adventure|Animation|Children'] 
title: ['Antz (1998)'], genres: ['Adventure|Animation|Children|Comedy|Fantasy'] 
title: ['Jaws (1975)'], genres: ['Action|Horror'] 
title: ['Stargate (1994)'], genres: ['Action|Adventure|Sci-Fi'] 
title: ['Star Wars: Episode I - The Phantom Menace (1999)'], genres: ['Action|Adventure|Sci-Fi'] 
title: ['Honey, I Shrunk the Kids (1989)'], genres: ['Adventure|Children|

In [ ]:
data = pd.read_csv(path[0])
user_data = data[data[data_frame[0]] == excluded_user]
user_data_5stars = user_data[user_data[data_frame[2]] == 5.0]  # Filter the data with a rating of 5.0r_user_data[r_user_data[data_frame[2]] == 5.0]  # Filter the data with a rating of 5.0
# print(user_data_5stars.head(10))
data = user_data_5stars.head(15)
print(data['movieId'].values)
moviedIds = data['movieId'].values

[ 47  50 101 151 157 163 216 231 260 333 362 457 527 553 596]


In [ ]:
df = pd.read_csv(path[1])

titles = []
genres = []

for i in moviedIds:
    title = df[df['movieId'] == i]['title']
    genre = df[df['movieId'] == i]['genres']
    titles.append(title)
    genres.append(genre)
for i in range(len(moviedIds)):
  print(f"title: {titles[i].values}, genres: {genres[i].values} ")

"""
title: ['Seven (a.k.a. Se7en) (1995)'], genres: ['Mystery|Thriller']  ->
title: ['Usual Suspects, The (1995)'], genres: ['Crime|Mystery|Thriller']  ->
title: ['Bottle Rocket (1996)'], genres: ['Adventure|Comedy|Crime|Romance'] 
title: ['Rob Roy (1995)'], genres: ['Action|Drama|Romance|War'] 
title: ['Canadian Bacon (1995)'], genres: ['Comedy|War'] 
title: ['Desperado (1995)'], genres: ['Action|Romance|Western'] 
title: ['Billy Madison (1995)'], genres: ['Comedy'] 
title: ['Dumb & Dumber (Dumb and Dumber) (1994)'], genres: ['Adventure|Comedy'] 
title: ['Star Wars: Episode IV - A New Hope (1977)'], genres: ['Action|Adventure|Sci-Fi'] ->
title: ['Tommy Boy (1995)'], genres: ['Comedy'] 
title: ['Jungle Book, The (1994)'], genres: ['Adventure|Children|Romance'] 
title: ['Fugitive, The (1993)'], genres: ['Thriller'] 
title: ["Schindler's List (1993)"], genres: ['Drama|War'] 
title: ['Tombstone (1993)'], genres: ['Action|Drama|Western'] 
title: ['Pinocchio (1940)'], genres: ['Animation|Children|Fantasy|Musical'] 
"""

title: ['Seven (a.k.a. Se7en) (1995)'], genres: ['Mystery|Thriller'] 
title: ['Usual Suspects, The (1995)'], genres: ['Crime|Mystery|Thriller'] 
title: ['Bottle Rocket (1996)'], genres: ['Adventure|Comedy|Crime|Romance'] 
title: ['Rob Roy (1995)'], genres: ['Action|Drama|Romance|War'] 
title: ['Canadian Bacon (1995)'], genres: ['Comedy|War'] 
title: ['Desperado (1995)'], genres: ['Action|Romance|Western'] 
title: ['Billy Madison (1995)'], genres: ['Comedy'] 
title: ['Dumb & Dumber (Dumb and Dumber) (1994)'], genres: ['Adventure|Comedy'] 
title: ['Star Wars: Episode IV - A New Hope (1977)'], genres: ['Action|Adventure|Sci-Fi'] 
title: ['Tommy Boy (1995)'], genres: ['Comedy'] 
title: ['Jungle Book, The (1994)'], genres: ['Adventure|Children|Romance'] 
title: ['Fugitive, The (1993)'], genres: ['Thriller'] 
title: ["Schindler's List (1993)"], genres: ['Drama|War'] 
title: ['Tombstone (1993)'], genres: ['Action|Drama|Western'] 
title: ['Pinocchio (1940)'], genres: ['Animation|Children|Fanta

'\ntitle: [\'Seven (a.k.a. Se7en) (1995)\'], genres: [\'Mystery|Thriller\']  ->\ntitle: [\'Usual Suspects, The (1995)\'], genres: [\'Crime|Mystery|Thriller\']  ->\ntitle: [\'Bottle Rocket (1996)\'], genres: [\'Adventure|Comedy|Crime|Romance\'] \ntitle: [\'Rob Roy (1995)\'], genres: [\'Action|Drama|Romance|War\'] \ntitle: [\'Canadian Bacon (1995)\'], genres: [\'Comedy|War\'] \ntitle: [\'Desperado (1995)\'], genres: [\'Action|Romance|Western\'] \ntitle: [\'Billy Madison (1995)\'], genres: [\'Comedy\'] \ntitle: [\'Dumb & Dumber (Dumb and Dumber) (1994)\'], genres: [\'Adventure|Comedy\'] \ntitle: [\'Star Wars: Episode IV - A New Hope (1977)\'], genres: [\'Action|Adventure|Sci-Fi\'] ->\ntitle: [\'Tommy Boy (1995)\'], genres: [\'Comedy\'] \ntitle: [\'Jungle Book, The (1994)\'], genres: [\'Adventure|Children|Romance\'] \ntitle: [\'Fugitive, The (1993)\'], genres: [\'Thriller\'] \ntitle: ["Schindler\'s List (1993)"], genres: [\'Drama|War\'] \ntitle: [\'Tombstone (1993)\'], genres: [\'Action|Dr

In [ ]:
for i in range(K):
  if(lib_titles[i].values  == titles[i].values):
      print(f"title: {lib_titles[i].values}, genres: {lib_genres[i].values} ")

신규 데이터를 어떻게 예측
- 이미 평가를 한 유저
- 콜드 스타트 유저
데이터 전처리 
추천에서 정확도를 어떻게 평가하는지 기준이 필요
- 데이터에 따라 틀림

유사도 모델기반